In [9]:
# Read Data
f = open("data/bezdekIris.data", "r")
fData = f.readlines()
lData = []

for sIter in fData:
    if (sIter[-1] == "\n"):
        sIter = sIter[:-1]
    if (sIter == ""):
        continue
    lData.append(sIter.split(","))

print(len(lData[0]))

5


In [10]:
# Create HashMaps
lDicts = [dict() for x in range(len(lData[0]))] # array/list of dictionaries (Hashmaps)
for i in range(0, len(lData[0])):
    dFeatures = {} # Empty hashmaps
    for j in range(0, len(lData)):
        if lData[j][i] in dFeatures.keys():
            dFeatures[lData[j][i]] = dFeatures[lData[j][i]] + 1
        else:
            dFeatures[lData[j][i]] = 1
    lDicts[i] = dFeatures

In [20]:
from copy import deepcopy
nIcebergCondn = 10
lData1 = deepcopy(lData)
for i in range(0, len(lData[0])):
    dInterest = lDicts[i]
    for j in range(0, len(lData)):
        if (dInterest[lData[j][i]] < nIcebergCondn):
            lData1[j][i] = '*';
print(lData1)

[['*', '*', '1.4', '0.2', 'Iris-setosa'], ['*', '3.0', '1.4', '0.2', 'Iris-setosa'], ['*', '3.2', '*', '0.2', 'Iris-setosa'], ['*', '3.1', '1.5', '0.2', 'Iris-setosa'], ['5.0', '*', '1.4', '0.2', 'Iris-setosa'], ['*', '*', '*', '*', 'Iris-setosa'], ['*', '3.4', '1.4', '*', 'Iris-setosa'], ['5.0', '3.4', '1.5', '0.2', 'Iris-setosa'], ['*', '2.9', '1.4', '0.2', 'Iris-setosa'], ['*', '3.1', '1.5', '*', 'Iris-setosa'], ['*', '*', '1.5', '0.2', 'Iris-setosa'], ['*', '3.4', '*', '0.2', 'Iris-setosa'], ['*', '3.0', '1.4', '*', 'Iris-setosa'], ['*', '3.0', '*', '*', 'Iris-setosa'], ['*', '*', '*', '0.2', 'Iris-setosa'], ['*', '*', '1.5', '*', 'Iris-setosa'], ['*', '*', '*', '*', 'Iris-setosa'], ['*', '*', '1.4', '*', 'Iris-setosa'], ['*', '*', '*', '*', 'Iris-setosa'], ['*', '*', '1.5', '*', 'Iris-setosa'], ['*', '3.4', '*', '0.2', 'Iris-setosa'], ['*', '*', '1.5', '*', 'Iris-setosa'], ['*', '*', '*', '0.2', 'Iris-setosa'], ['*', '*', '*', '*', 'Iris-setosa'], ['*', '3.4', '*', '0.2', 'Iris-se

In [22]:
lData2 = deepcopy(lData1)
for i in range(0, len(lData2)):
    lData2[i] = tuple(lData2[i])
lData2

[('*', '*', '1.4', '0.2', 'Iris-setosa'),
 ('*', '3.0', '1.4', '0.2', 'Iris-setosa'),
 ('*', '3.2', '*', '0.2', 'Iris-setosa'),
 ('*', '3.1', '1.5', '0.2', 'Iris-setosa'),
 ('5.0', '*', '1.4', '0.2', 'Iris-setosa'),
 ('*', '*', '*', '*', 'Iris-setosa'),
 ('*', '3.4', '1.4', '*', 'Iris-setosa'),
 ('5.0', '3.4', '1.5', '0.2', 'Iris-setosa'),
 ('*', '2.9', '1.4', '0.2', 'Iris-setosa'),
 ('*', '3.1', '1.5', '*', 'Iris-setosa'),
 ('*', '*', '1.5', '0.2', 'Iris-setosa'),
 ('*', '3.4', '*', '0.2', 'Iris-setosa'),
 ('*', '3.0', '1.4', '*', 'Iris-setosa'),
 ('*', '3.0', '*', '*', 'Iris-setosa'),
 ('*', '*', '*', '0.2', 'Iris-setosa'),
 ('*', '*', '1.5', '*', 'Iris-setosa'),
 ('*', '*', '*', '*', 'Iris-setosa'),
 ('*', '*', '1.4', '*', 'Iris-setosa'),
 ('*', '*', '*', '*', 'Iris-setosa'),
 ('*', '*', '1.5', '*', 'Iris-setosa'),
 ('*', '3.4', '*', '0.2', 'Iris-setosa'),
 ('*', '*', '1.5', '*', 'Iris-setosa'),
 ('*', '*', '*', '0.2', 'Iris-setosa'),
 ('*', '*', '*', '*', 'Iris-setosa'),
 ('*', '3.

In [27]:
dFeatures = {}

for i in range(0, len(lData2)):
    if lData2[i] in dFeatures.keys():
        dFeatures[lData2[i]] = dFeatures[lData2[i]] + 1
    else:
        dFeatures[lData2[i]] = 1
dFeatures

{('*', '*', '1.4', '0.2', 'Iris-setosa'): 2,
 ('*', '3.0', '1.4', '0.2', 'Iris-setosa'): 1,
 ('*', '3.2', '*', '0.2', 'Iris-setosa'): 3,
 ('*', '3.1', '1.5', '0.2', 'Iris-setosa'): 2,
 ('5.0', '*', '1.4', '0.2', 'Iris-setosa'): 2,
 ('*', '*', '*', '*', 'Iris-setosa'): 6,
 ('*', '3.4', '1.4', '*', 'Iris-setosa'): 1,
 ('5.0', '3.4', '1.5', '0.2', 'Iris-setosa'): 1,
 ('*', '2.9', '1.4', '0.2', 'Iris-setosa'): 1,
 ('*', '3.1', '1.5', '*', 'Iris-setosa'): 1,
 ('*', '*', '1.5', '0.2', 'Iris-setosa'): 3,
 ('*', '3.4', '*', '0.2', 'Iris-setosa'): 3,
 ('*', '3.0', '1.4', '*', 'Iris-setosa'): 2,
 ('*', '3.0', '*', '*', 'Iris-setosa'): 1,
 ('*', '*', '*', '0.2', 'Iris-setosa'): 4,
 ('*', '*', '1.5', '*', 'Iris-setosa'): 4,
 ('*', '*', '1.4', '*', 'Iris-setosa'): 2,
 ('5.0', '3.0', '*', '0.2', 'Iris-setosa'): 1,
 ('5.0', '3.4', '*', '*', 'Iris-setosa'): 1,
 ('*', '3.4', '1.4', '0.2', 'Iris-setosa'): 1,
 ('*', '3.1', '*', '0.2', 'Iris-setosa'): 1,
 ('*', '3.4', '1.5', '*', 'Iris-setosa'): 1,
 ('5.0

In [31]:
lKeys = sorted(dFeatures.keys())
lKeys

[('*', '*', '*', '*', 'Iris-setosa'),
 ('*', '*', '*', '*', 'Iris-versicolor'),
 ('*', '*', '*', '*', 'Iris-virginica'),
 ('*', '*', '*', '0.2', 'Iris-setosa'),
 ('*', '*', '*', '1.3', 'Iris-versicolor'),
 ('*', '*', '*', '1.5', 'Iris-versicolor'),
 ('*', '*', '*', '1.5', 'Iris-virginica'),
 ('*', '*', '*', '1.8', 'Iris-virginica'),
 ('*', '*', '1.4', '*', 'Iris-setosa'),
 ('*', '*', '1.4', '0.2', 'Iris-setosa'),
 ('*', '*', '1.5', '*', 'Iris-setosa'),
 ('*', '*', '1.5', '0.2', 'Iris-setosa'),
 ('*', '2.8', '*', '*', 'Iris-versicolor'),
 ('*', '2.8', '*', '*', 'Iris-virginica'),
 ('*', '2.8', '*', '1.3', 'Iris-versicolor'),
 ('*', '2.8', '*', '1.5', 'Iris-versicolor'),
 ('*', '2.8', '*', '1.5', 'Iris-virginica'),
 ('*', '2.8', '*', '1.8', 'Iris-virginica'),
 ('*', '2.9', '*', '*', 'Iris-versicolor'),
 ('*', '2.9', '*', '1.3', 'Iris-versicolor'),
 ('*', '2.9', '*', '1.5', 'Iris-versicolor'),
 ('*', '2.9', '*', '1.8', 'Iris-virginica'),
 ('*', '2.9', '1.4', '0.2', 'Iris-setosa'),
 ('*', 

In [51]:
#Tree class
class Tree(object):
    def __init__(node, label = "all", children = None, count = 0):
        node.name = label
        node.children = []
        node.count = count
        if (children != None):
            for i in (0, len(children)):
                node.add_node(children[i])
    
    def add_node(node, child):
        node.children.append(child)
    
    def find(node, label = str("")):
        for i in range(0, len(node.children)):
            if (node.children[i].name == label):
                return node.children[i]
        return None
    def update_count(node, count = 0):
        node.count += count
        

In [53]:
#create star tree
star_tree = Tree()
for i in range (0, len(lKeys)):
    print(lKeys[i][0])
    tLeaf = star_tree.find(label = lKeys[i][0])  
    if (tLeaf == None):
        tLeaf = Tree(label = lKeys[i][0], count = dFeatures[lKeys[i]])
        star_tree.add_node(tLeaf)
    else:
        tLeaf.update_count(count = dFeatures[lKeys[i]])
    for j in range (1, len(lKeys[0])):
        tNew = tLeaf.find(lKeys[i][0])
        if (tNew == None):
            tNew = Tree(label = lKeys[i][0], count = dFeatures[lKeys[i]])
            
    
    

*
*


AttributeError: 'Tree' object has no attribute 'label'

In [44]:
lKeys[0][1]

'*'